In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


here the objective is to fit the logit graph model to multiple different graph domains and then visualize the original vs fitted graphs

In [4]:

import sys
import os
sys.path.append('../..')

#Graph imports
import src.graph as graph
import src.logit_estimator as estimator
import src.utils as utils
import src.model_selection as model_selection
import src.gic as gic
import src.param_estimator as pe
import src.graph as graph
import src.model_selection as ms

# usual imports
import matplotlib.pyplot as plt
import pickle
import math
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import gaussian_kde
import numpy as np
import pandas as pd
import seaborn as sns
import gc
import random
import networkx as nx
from numpy import errstate

from IPython.display import display
from pyvis.network import Network
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [5]:
FOLDER_OUTPUT = f'runs/fitted_graphs_comparison_gplus1'
out_files = os.listdir(FOLDER_OUTPUT)
out_files = sorted(out_files, key=lambda x: os.path.getctime(os.path.join(FOLDER_OUTPUT, x)), reverse=True)

out_files

['comparators_104672614700283598130.pkl',
 'comparators_114336431216099933033.pkl',
 'comparators_104105354262797387583.pkl',
 'comparators_103503116383846951534.pkl',
 'comparators_108404515213153345305.pkl',
 'comparators_103251633033550231172.pkl',
 'comparators_100466178325794757407.pkl',
 'comparators_110614416163543421878.pkl',
 'comparators_100518419853963396365.pkl',
 'comparators_113597493946570654755.pkl',
 'comparators_114104634069486127920.pkl',
 'comparators_117503822947457399073.pkl',
 'comparators_100637660947564674695.pkl',
 'comparators_101499880233887429402.pkl',
 'comparators_111213696402662884531.pkl',
 'comparators_117412175333096244275.pkl',
 'comparators_118255645714452180374.pkl',
 'comparators_114054672576929802335.pkl',
 'comparators_117798157258572080176.pkl',
 'comparators_103241736833663734962.pkl',
 'comparators_110232479818136355682.pkl',
 'comparators_117866881767579360121.pkl',
 'comparators_113455290791279442483.pkl',
 'comparators_10261586334441046775

In [6]:
import pickle
#pick first id from out_files
id = out_files[0].split('_')[1]

with open(f'{FOLDER_OUTPUT}/comparators_{id}', 'rb') as f:
    comparators = pickle.load(f)


In [7]:
len(comparators)

24

In [8]:
dfs = []
for i in range(len(comparators)):
    dfs.append(comparators[i].summary_df)

In [9]:
dfs[1]

,graph_filename,model,gic_value,param,fit_success,nodes,edges,density,avg_clustering,avg_path_length,diameter,assortativity,num_components,largest_component_size
0,113455290791279442483.edges,Original,NaN,N/A,True,108,2352,0.407061,0.690156,1.598477,3,-0.244844,1,108
1,113455290791279442483.edges,LG,0.014621,"d=2, sigma=-170.9213",True,108,2125,0.367774,0.368070,1.632226,2,-0.013983,1,108
2,113455290791279442483.edges,ER,0.284386,0.2,True,108,1201,0.207857,0.208110,1.801488,3,-0.028331,1,108
3,113455290791279442483.edges,BA,0.702993,5.0,True,108,515,0.089131,0.179604,2.268951,4,-0.076620,1,108
4,113455290791279442483.edges,GRG,1.726044,1.0,True,108,5684,0.983731,0.985327,1.016269,2,-0.045163,1,108
5,113455290791279442483.edges,WS,1.760145,20.178889,True,108,2376,0.411215,0.411166,1.588785,2,-0.015830,1,108


In [10]:
pd.concat(dfs)

,graph_filename,model,gic_value,param,fit_success,nodes,edges,density,avg_clustering,avg_path_length,diameter,assortativity,num_components,largest_component_size
0,102615863344410467759.edges,Original,NaN,N/A,True,102,1169,0.226946,0.597790,1.791303,3,-0.358295,1,102
1,102615863344410467759.edges,LG,0.426997,"d=2, sigma=-32.4977",True,102,1163,0.225781,0.226596,1.779072,3,-0.035959,1,102
2,102615863344410467759.edges,ER,0.282287,0.2,True,102,1050,0.203844,0.197667,1.804698,3,-0.006689,1,102
3,102615863344410467759.edges,BA,0.346601,5.0,True,102,485,0.094156,0.154491,2.268492,4,-0.044186,1,102
4,102615863344410467759.edges,WS,1.391627,20.157778,True,102,1122,0.217822,0.212017,1.788391,3,-0.037944,1,102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,104672614700283598130.edges,LG,0.192415,"d=1, sigma=-5.3560",True,475,16369,0.145405,0.145151,1.854630,3,-0.014153,1,475
2,104672614700283598130.edges,ER,0.196368,0.136667,True,475,15286,0.135785,0.135127,1.864348,3,-0.017776,1,475
3,104672614700283598130.edges,BA,0.752832,5.0,True,475,2350,0.020875,0.069393,2.754315,4,-0.050471,1,475
4,104672614700283598130.edges,GRG,1.309566,1.0,True,475,109081,0.968963,0.973095,1.031037,2,-0.044677,1,475


In [11]:
# Create a DataFrame with GIC values for each model and graph
gic_df = pd.DataFrame()

for df in dfs:
    graph_name = df['graph_filename'].iloc[0]
    # Get GIC values for each model (excluding Original which has NaN)
    model_gics = df[df['model'] != 'Original'].set_index('model')['gic_value']
    gic_df[graph_name] = model_gics

# Transpose so graphs are rows and models are columns
gic_df = gic_df.T
gic_df

model,LG,ER,BA,WS,GRG
102615863344410467759.edges,0.426997,0.282287,0.346601,1.391627,5.096360
113455290791279442483.edges,0.014621,0.284386,0.702993,1.760145,1.726044
117866881767579360121.edges,0.421547,1.937106,0.506701,1.102526,7.321368
110232479818136355682.edges,0.737491,1.741964,0.493616,1.026964,7.793429
103241736833663734962.edges,1.420944,1.397497,0.703680,1.178889,6.662467
117798157258572080176.edges,1.456703,3.177212,1.488068,0.922381,7.894253
114054672576929802335.edges,1.025417,1.109970,1.264286,1.502292,2.741588
118255645714452180374.edges,0.099664,0.165434,0.733533,1.765583,1.225340
117412175333096244275.edges,1.191174,3.690451,1.224110,0.805347,8.957375
111213696402662884531.edges,0.152789,0.145423,0.606795,1.648672,2.223929


In [12]:
# Create a dictionary to store rankings for each graph
rankings = {}

# Go through each df in dfs
for df in dfs:
    graph_name = df['graph_filename'].iloc[0]  # Get graph filename
    
    # Sort models by GIC value (excluding 'Original' which has NaN GIC)
    model_ranks = df[df['model'] != 'Original'].sort_values('gic_value')['model'].tolist()
    
    # Create rankings dictionary for this graph (1 is best since lowest GIC is best)
    rankings[graph_name] = {
        model: rank+1 for rank, model in enumerate(model_ranks)
    }
    
    # Add nodes and edges from Original model
    original_row = df[df['model'] == 'Original'].iloc[0]
    rankings[graph_name]['nodes'] = original_row['nodes']
    rankings[graph_name]['edges'] = original_row['edges']

# Get unique models (excluding 'Original')
models = sorted(list(set([model for df in dfs for model in df['model'].unique() if model != 'Original'])))

# Create DataFrame with rankings
ranking_df = pd.DataFrame(rankings).T

# Ensure all models are present as columns
for model in models:
    if model not in ranking_df.columns:
        ranking_df[model] = np.nan

# Keep only the model columns in a consistent order, plus nodes and edges
ranking_df = ranking_df[models + ['nodes', 'edges']]

print("Rankings by GIC value (lower is better):")
display(ranking_df)


Rankings by GIC value (lower is better):


,BA,ER,GRG,LG,WS,nodes,edges
102615863344410467759.edges,2,1,5,3,4,102,1169
113455290791279442483.edges,3,2,4,1,5,108,2352
117866881767579360121.edges,2,4,5,1,3,117,720
110232479818136355682.edges,1,4,5,2,3,120,1026
103241736833663734962.edges,1,3,5,4,2,131,1342
117798157258572080176.edges,3,4,5,2,1,168,892
114054672576929802335.edges,3,2,5,1,4,175,4138
118255645714452180374.edges,3,2,4,1,5,215,6189
117412175333096244275.edges,3,4,5,2,1,252,946
111213696402662884531.edges,3,1,5,2,4,258,7624


In [13]:
ranking_df.mean(axis=0)

BA          2.291667
ER          2.250000
GRG         4.750000
LG          2.083333
WS          3.625000
nodes     281.916667
edges    5708.708333
dtype: float64

In [14]:
ranking_df['nodes'].describe()

count     24.000000
mean     281.916667
std      119.154601
min      102.000000
25%      173.250000
50%      301.500000
75%      351.250000
max      475.000000
Name: nodes, dtype: float64